# Dependencies

In [1]:
# !pip install -r requirements.txt

In [2]:
# !pip freeze > requirements.txt

In [3]:
!pip freeze

absl-py==1.4.0
anyio==3.6.2
appnope==0.1.3
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
astunparse==1.6.3
attrs==22.2.0
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
cachetools==5.3.0
certifi==2022.12.7
cffi==1.15.1
charset-normalizer==3.1.0
click==8.1.3
comm==0.1.2
contourpy==1.0.7
cycler==0.11.0
debugpy==1.6.6
decorator==5.1.1
defusedxml==0.7.1
executing==1.2.0
fastjsonschema==2.16.3
Flask==2.2.3
flatbuffers==1.12
fonttools==4.39.0
fqdn==1.5.1
gast==0.4.0
google-auth==2.16.2
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.51.3
h5py==3.8.0
idna==3.4
importlib-metadata==6.0.0
importlib-resources==5.12.0
ipykernel==6.21.3
ipython==8.10.0
ipython-genutils==0.2.0
ipywidgets==8.0.4
isoduration==20.11.0
itsdangerous==2.1.2
jedi==0.18.2
Jinja2==3.1.2
joblib==1.2.0
jsonpointer==2.3
jsonschema==4.17.3
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.6.3
jupyter_client==8.0.3
jupyter_core==5.2.0
jupyter_server==2.4.0
jupyter_server_termi

# PreProcessing

## Dependencies

In [4]:
import matplotlib as mlp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


## Convert data into numpy array

In [5]:
# import pathlib


test_mask = [] 
test_no_mask = []
# I would do this in a fancy os.loop way but I was using google colab and didn't really want to mess with the function too much
train_data_mask_dir="New_Masks_Dataset/Train/Mask"
train_data_no_mask_dir="New_Masks_Dataset/Train/Non_Mask"
val_data_mask_dir = 'New_Masks_Dataset/Validation/Mask'
val_data_no_mask_dir = 'New_Masks_Dataset/Validation/Non_Mask'
test_data_mask_dir="New_Masks_Dataset/Test/Mask"
test_data_no_mask_dir="New_Masks_Dataset/Test/Non_Mask"


In [6]:
# print(train_imagePaths_mask)

In [7]:
# https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
# https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
import cv2
# from google.colab.patches import cv2_imshow
import os

IMAGE_HEIGHT  = 100
IMAGE_WIDTH = 100
NUM_CLASSES = 2


input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT)


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        # cv2_imshow(img)
        # Has to be like this because of Python Ambiguity
        if img is not None:
            grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(grayscale, input_shape)
            resized = np.reshape(resized, input_shape[0] * input_shape[1])
            images.append(resized)
    return images


In [8]:
# Train
train_mask = load_images_from_folder(train_data_mask_dir)
train_no_mask = load_images_from_folder(train_data_no_mask_dir)
# Val
val_mask = load_images_from_folder(val_data_mask_dir)
val_no_mask = load_images_from_folder(val_data_no_mask_dir)
# Test
test_mask = load_images_from_folder(test_data_mask_dir)
test_no_mask = (load_images_from_folder(test_data_no_mask_dir))

In [9]:
train_mask += val_mask
train_no_mask += val_no_mask

In [10]:
y_train_mask = [1] * len(train_mask)
y_train_no_mask = [0] * len(train_no_mask)

y_val_mask = [1] * len(val_mask)
y_val_no_mask = [0] * len(val_no_mask)

y_test_mask = [1] * len(test_mask)
y_test_no_mask = [0] * len(test_no_mask)


In [11]:
df_train = pd.DataFrame({'label': y_train_mask,'mask_data': train_mask})

In [12]:
df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_35484/1233198709.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(pd.DataFrame({'label': y_train_no_mask, 'mask_data': train_no_mask}), ignore_index=True)


In [13]:
df_train = df_train.sample(frac = 1)
df_train

,label,mask_data
206,1,"[184, 186, 183, 187, 185, 182, 181, 175, 175, ..."
489,0,"[174, 171, 157, 127, 115, 116, 135, 136, 139, ..."
492,0,"[157, 159, 131, 138, 130, 178, 211, 220, 229, ..."
30,1,"[176, 168, 195, 152, 151, 146, 146, 132, 119, ..."
254,1,"[42, 120, 238, 233, 226, 219, 214, 212, 205, 2..."
...,...,...
106,1,"[42, 42, 43, 43, 45, 44, 43, 43, 44, 44, 44, 4..."
499,0,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
287,1,"[94, 195, 97, 100, 101, 104, 105, 109, 105, 84..."
752,0,"[178, 200, 206, 209, 207, 215, 208, 197, 180, ..."


In [14]:
X_train = df_train.iloc[:, 1:] / 255.0
X_train = pd.DataFrame(X_train['mask_data'].tolist())
y_train = df_train.iloc[:, :1]

# .astype('float32')

In [15]:
df_val = pd.DataFrame({'label': y_val_mask,'mask_data': val_mask})
df_val = df_val.append(pd.DataFrame({'label': y_val_no_mask, 'mask_data': val_no_mask}), ignore_index=True)

df_val = df_val.sample(frac = 1)
df_val

/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_35484/3146285510.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_val = df_val.append(pd.DataFrame({'label': y_val_no_mask, 'mask_data': val_no_mask}), ignore_index=True)


,label,mask_data
28,1,"[65, 52, 56, 59, 58, 58, 59, 59, 59, 59, 51, 4..."
285,0,"[165, 183, 196, 205, 183, 201, 253, 253, 255, ..."
42,1,"[71, 71, 71, 71, 71, 71, 72, 72, 71, 71, 72, 7..."
253,0,"[42, 41, 39, 31, 29, 32, 24, 36, 26, 20, 28, 2..."
149,0,"[29, 28, 27, 28, 30, 29, 28, 28, 28, 27, 25, 2..."
...,...,...
212,0,"[225, 223, 221, 222, 223, 224, 223, 223, 224, ..."
134,1,"[195, 196, 184, 191, 185, 180, 136, 173, 199, ..."
126,1,"[17, 16, 19, 15, 16, 19, 21, 18, 24, 25, 28, 3..."
201,0,"[157, 129, 142, 156, 167, 176, 173, 146, 129, ..."


In [16]:
X_val = df_val.iloc[:, 1:] / 255.0
X_val = pd.DataFrame(X_val['mask_data'].tolist())
y_val = df_val.iloc[:,:1]


In [17]:
df_test = pd.DataFrame({'label': y_test_mask,'mask_data': test_mask})
df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)

df_test = df_test.sample(frac = 1)
df_test


/var/folders/21/4ybwj4pj54x1sz2qhfgy1b9m0000gn/T/ipykernel_35484/4284791088.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(pd.DataFrame({'label': y_test_no_mask, 'mask_data': test_no_mask}), ignore_index=True)


,label,mask_data
67,0,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ..."
66,0,"[98, 99, 99, 98, 99, 99, 99, 99, 99, 100, 100,..."
87,0,"[217, 222, 221, 220, 221, 223, 224, 225, 225, ..."
77,0,"[179, 190, 187, 180, 188, 182, 186, 181, 185, ..."
33,1,"[136, 118, 101, 115, 107, 73, 241, 241, 241, 2..."
...,...,...
49,0,"[125, 126, 125, 126, 126, 125, 126, 126, 126, ..."
40,1,"[139, 105, 89, 246, 230, 232, 237, 241, 242, 2..."
29,1,"[157, 127, 150, 160, 145, 145, 71, 112, 71, 54..."
78,0,"[212, 211, 213, 214, 216, 219, 221, 224, 226, ..."


In [18]:
X_test = df_test.iloc[:, 1:] / 255.0
X_test = pd.DataFrame(X_test['mask_data'].tolist())
y_test = df_test.iloc[:, :1]


In [19]:
x_train = X_train.values.reshape(-1,100,100)
x_val = X_val.values.reshape(-1,100,100)
x_test = X_test.values.reshape(-1,100,100)

In [20]:
batch_size = 128
epochs = 15
# input image dimensions
input_shape = (100, 100, 1)

y = y_train #use later for the classifiers
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_val = keras.utils.to_categorical(y_val, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)



In [21]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(901, 100, 100)
(302, 100, 100)
(99, 100, 100)


# Running Model

## KNN (K_Nearest_Neighbor Classifier)

In [22]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn import metrics
# from sklearn.model_selection import GridSearchCV
# k_range = list(range(1, 30))
# param_grid = {'n_neighbors': k_range, 
#               'weights': ['uniform', 'distance']}
# knn = KNeighborsClassifier()
# grid_search = GridSearchCV(knn, param_grid, cv=5, verbose = 1)
# grid_search.fit(X_train, y_train)
# y_pred = grid_search.predict(X_test)
# metrics.f1_score(y_test, y_pred, average='weighted')
# print("Best Parameters:", grid_search.best_params_)
# print("Accuracy Score:", grid_search.best_score_)

In [23]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score

# lin_classifier = SGDClassifier()
# lin_classifier.fit(X_train, y)
# y_pred = lin_classifier.predict(X_test)

# accuracy_score(y_test, y_pred)


In [24]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# # Train and evaluate several classifiers
# classifiers = [
#     LogisticRegression(),
#     SVC(),
# ]
# y= y['label']

# for clf in classifiers:
#     clf.fit(X_train, y)
#     y_pred = clf.predict(X_test)
#     print(type(clf).__name__)
#     print("Accuracy: {:.2f}".format(accuracy_score(y_test, y_pred)))
#     print("Precision: {:.2f}".format(precision_score(y_test, y_pred, average='weighted')))
#     print("Recall: {:.2f}".format(recall_score(y_test, y_pred, average='weighted')))
#     print("F1-score: {:.2f}".format(f1_score(y_test, y_pred, average='weighted')))
#     print()

In [25]:
# from keras.preprocessing import image

# train_data_mask_dir="New_Masks_Dataset/Train"
# val_data_mask_dir = "New_Masks_Dataset/Validation"
# test_data_mask_dir="New_Masks_Dataset/Test"

# train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True,)
# test_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow_from_directory(train_data_mask_dir, target_size=(100, 100), batch_size=32, class_mode='binary')
# validation_generator = test_datagen.flow_from_directory(val_data_mask_dir, target_size=(100, 100), batch_size=32, class_mode='binary')


## CNN (Convolutional neural network)

In [26]:

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/RandomFlip

# The parameters I chose here were also from a project I did earlier, I'll link it here
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomZoom(-.1, .1),
        keras.layers.RandomFlip(mode='horizontal_and_vertical'),
        
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation="sigmoid"),
    ]
)

model.summary()

2023-04-03 11:07:56.309208: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_rotation (RandomRota  (None, 100, 100, 1)      0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 100, 100, 1)       0         
                                                                 
 random_flip (RandomFlip)    (None, 100, 100, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                      

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True),
             EarlyStopping(monitor='val_loss', patience=7)]

model.fit(x_train, y_train, epochs=epochs,
                    callbacks=callbacks,
                    validation_data=(x_test, y_test))


Epoch 1/15
29/29 [==============================] - 23s 727ms/step - loss: 0.6197 - accuracy: 0.6404 - val_loss: 0.5357 - val_accuracy: 0.7273
Epoch 2/15
29/29 [==============================] - 23s 780ms/step - loss: 0.4882 - accuracy: 0.7691 - val_loss: 0.4554 - val_accuracy: 0.7778
Epoch 3/15
29/29 [==============================] - 21s 718ms/step - loss: 0.4643 - accuracy: 0.7980 - val_loss: 0.3894 - val_accuracy: 0.7980
Epoch 4/15
29/29 [==============================] - 20s 687ms/step - loss: 0.4723 - accuracy: 0.7802 - val_loss: 0.3697 - val_accuracy: 0.8283
Epoch 5/15
29/29 [==============================] - 19s 660ms/step - loss: 0.4438 - accuracy: 0.8013 - val_loss: 0.3345 - val_accuracy: 0.8384
Epoch 6/15
29/29 [==============================] - 20s 691ms/step - loss: 0.4339 - accuracy: 0.8191 - val_loss: 0.3213 - val_accuracy: 0.8687
Epoch 7/15
29/29 [==============================] - 21s 723ms/step - loss: 0.4132 - accuracy: 0.8147 - val_loss: 0.2976 - val_accuracy: 0.8788

In [33]:
model.save('best_model.h5')

In [29]:
# model = keras.models.load_model('./best_model.h5')

In [30]:
model.evaluate(x_test, y_test)

4/4 [==============================] - 1s 109ms/step - loss: 0.2387 - accuracy: 0.8889


[0.23869213461875916, 0.8888888955116272]

In [31]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.23869213461875916
Test accuracy: 0.8888888955116272


In [32]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

print(classification_report(y_true_labels, y_pred_labels))
print(confusion_matrix(y_true_labels, y_pred_labels))

4/4 [==============================] - 1s 129ms/step
              precision    recall  f1-score   support

           0       0.91      0.86      0.89        50
           1       0.87      0.92      0.89        49

    accuracy                           0.89        99
   macro avg       0.89      0.89      0.89        99
weighted avg       0.89      0.89      0.89        99

[[43  7]
 [ 4 45]]
